## Integrate the particles where they feel all NBody forces

In [1]:
import tstrippy
from gcs import path_handler as ph
import gcs
import h5py

In [11]:
montecarlokey = "monte-carlo-000"
GCnames=list(tstrippy.Parsers.baumgardtMWGCs().data['Cluster'][:])
Masses,rh_mes,RAes,DECes,Rsunes,RVes,mualphaes,mu_deltaes=gcs.data_extractors.extract_all_GC_observables(GCnames,montecarlokey)

In [14]:
# now get the positions at the begining of the orbit 
dir(ph)

['MonteCarloObservables',
 'ParticleDistribution',
 'Path',
 '_MonteCarloObservables',
 '_ParticleDistribution',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_temporary_gc_orbits',
 'current_script_directory',
 'file',
 'os',
 'paths',
 'paths_file_path',
 'temporary_gc_orbits',
 'yaml']